# COVIDvu

COVID-19 view various charts.

---
## Pull latest datasets from the JH CSSE GitHub repository

In [ ]:
%cd COVID-19

In [ ]:
%sx git fetch

In [ ]:
%sx git pull

In [ ]:
%cd ..

## Group confirmed cases by country

In [ ]:
import os

import pandas as pd

COVID19_PATH=os.path.join(os.path.join(os.getcwd(), 'COVID-19'), 'csse_covid_19_data/csse_covid_19_time_series')
CONFIRMED_FILE=os.path.join(COVID19_PATH, 'time_series_19-covid-Confirmed.csv')

CONFIRMED_FILE

In [ ]:
confirmedCases = pd.read_csv(CONFIRMED_FILE).groupby(['Country/Region']).sum()

confirmedCases

## Plot confirmed by date, filtered by country 

1. Set filter to the country name
1. Plot the time series from the beginning in January 2020 to latest value, Y axis = total count

In [19]:
import visualize

In [21]:
countryName = 'UK'

confirmedCasesCountryTS = confirmedCases[confirmedCases.columns[2:]].loc[countryName]

visualize.plotMultipleTimeSeries(x=confirmedCasesCountryTS.index,
                                 y=confirmedCasesCountryTS,
                                 name=countryName,
                                 yLabel='Total daily count',
                                 color='#690415',
                                 title=f'Confirmed cases in {countryName}'
                                )